In [1]:
#----- imports --------

import tqdm
import torch
from torch import nn
from torch.nn import functional as F
import wandb
import os
import tokenizers
from matplotlib import pyplot as plt
import numpy as np
import json
import random
import tqdm


device= 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_default_device(device)
assert device == 'cuda', "This notebook is not optimized for CPU"

config = {
    "learning_rate": 1e-3,
    "sae_learning_rate": 5e-5,
    "model_embedding_layer": 6,
    "eval_interval": 500,
    "max_iters": 60000, 
    "H": 32, # hidden dimension size
    "B": 64,
    "T": 256,
    "C": 256,
    "feedforward_factor": 3,
    "n_heads": 8,
    "n_layers": 12,
    "tokenizer_vocab_size": 2**13,
    "git_hash": os.popen("git rev-parse HEAD").read().strip()
}

# initial
for k,v in config.items():
    locals ()[k] = v


#wandb.init(
#    project = "tinystories",
#    config = config,
#)

In [2]:
class SparseAutoEncoder(nn.Module):
    def __init__(self, activations_dim, sparse_dim):
        super().__init__()
        self.activations_dim = activations_dim
        encoder_weight = torch.randn(activations_dim, sparse_dim)
        decoder_weight = torch.randn(sparse_dim, activations_dim)
        self.encoder_bias = nn.Parameter(torch.zeros(sparse_dim))
        self.decoder_bias = nn.Parameter(torch.zeros(activations_dim))
        self.sparse_dim = sparse_dim
        self.sparsity_penalty = 30

        # set the encoder_weight to have the activations dim to be normalized to have l2 norm randomly between 0.05 and 1
        direction_lengths = torch.rand(sparse_dim) * 0.95 + 0.05
        # normalize the encoder_weight along columns (dim -2) to have l2 norm of 1
        encoder_weight = F.normalize(encoder_weight, p=2, dim=0)
        # multiply the column norms by the direction_lengths
        encoder_weight = encoder_weight * direction_lengths
        # make the decoder weight be the transpose of the encoder_weight
        decoder_weight = torch.transpose(encoder_weight, 0, 1)

        self.encoder_weight = nn.Parameter(encoder_weight)
        self.decoder_weight = nn.Parameter(decoder_weight)



    def forward(self, x):
        # preprocessing normalization
        # now on average any embedding has euclidian length 1

        encoded = F.relu(x @ self.encoder_weight + self.encoder_bias) # all act. are positive
        decoded = encoded @ self.decoder_weight + self.decoder_bias

        reconstruction_l2_loss = F.mse_loss(x, decoded)

        # every row in the tall decoder matrix
        # is the "sum" of the total influence of a feature on the output
        # the l2 norm of that row is the "influence" of that feature on that output
        # calculate that, store as row
        decoder_l2 = torch.linalg.norm(self.decoder_weight, dim=-1)
        # the feature activation is the sparse activation * it's influence on output
        feature_activations = (encoded) * decoder_l2
        # sum of feature activations
        # divide by the batch size * sequence length
        # should work if there is no batch dimension
        if x.ndim == 3:
            batch_dim, sequence_dim, _ = x.shape
        elif x.ndim == 2:
            batch_dim = 1
            sequence_dim, _ = x.shape
        elif x.ndim == 1:
            batch_dim = 1
            sequence_dim = 1
        else:
            raise ValueError(f"x has {x.ndim} dimensions, but it should have 1, 2, or 3")
        
        sparsity_loss = torch.sum(feature_activations) * self.sparsity_penalty / (batch_dim * sequence_dim * self.sparse_dim)

        total_loss = reconstruction_l2_loss + sparsity_loss

        return {"encoded": encoded, "decoded": decoded, 'feature_activations': feature_activations, "reconstruction_loss": reconstruction_l2_loss, "sparsity_loss": sparsity_loss, "total_loss": total_loss}




# def test_sparse_autoencoder_sequence_independence():
#     sae = SparseAutoEncoder(100, 300)
#     input_embedding = torch.randn(2, 100)

#     input_embedding_modified = input_embedding.clone()
#     # modify the first in the sequence to be zeros
#     input_embedding_modified[0, :] = torch.zeros(100)
#     original_output = sae.forward(input_embedding)['decoded']
#     modified_output = sae.forward(input_embedding_modified)['decoded']

#     # make sure the last embedding in the sequence is the same, and the first is different
#     assert torch.all(original_output[-1, :] == modified_output[-1, :])
#     assert not torch.all(original_output[0, :] == modified_output[0, :])

# test_sparse_autoencoder_sequence_independence()

# def count_num_params_in_sae(sparse_dim_size):
#     sae = SparseAutoEncoder(C, sparse_dim_size)
#     num_params = sum(p.numel() for p in sae.parameters())
#     return num_params

# print("number of parameters:", count_num_params_in_sae(2**14))
# print(2**14)

# def reconstruction_training_run(embedding_size, sparse_dim_size, iters, eval_frequency):
#     def generate_tensor():
#         return torch.randn(embedding_size)
    
#     sae = SparseAutoEncoder(embedding_size, sparse_dim_size)
    
#     optimizer = torch.optim.Adam(sae.parameters(), lr=0.0005)
    
#     for i in range(iters):
#         optimizer.zero_grad()
#         input_tensor = generate_tensor()
#         output = sae.forward(input_tensor)
#         loss = output['total_loss']
#         loss.backward()
        
#         # Clip the gradient norm to 1
#         torch.nn.utils.clip_grad_norm_(sae.parameters(), max_norm=1)
        
#         optimizer.step()
        
#         if i % eval_frequency == 0:
#             avg_loss = 0
#             num_samples = 100  # Number of samples to average over
#             for _ in range(num_samples):
#                 input_tensor = generate_tensor()
#                 output = sae.forward(input_tensor)
#                 avg_loss += output['total_loss'].item()
#             avg_loss /= num_samples
#             print(f"Iteration {i}, Average Total Loss: {avg_loss}")
#     return sae

# reconstruction_training_run(C, 2**14, 10_000, 500)





In [3]:
sae = SparseAutoEncoder(C, 2**14)
optimizer = torch.optim.Adam(sae.parameters(), lr=sae_learning_rate)


In [4]:
def load_tensor(filepath):
    # load the .pt tensor
    tensor = torch.load(filepath)
    tensor = torch.cat(tensor, dim=0)
    tensor = tensor.to(device)
    return tensor
    

In [18]:
@torch.no_grad()
def estimate_sae_loss(eval_iters, tensor):
    sae_loss = 0
    sae_sparsity_loss = 0
    sae_reconstruction_loss = 0
    count = 0
    for i in range(0, eval_iters, B):
        count += 1
        start = i
        end = i+B
        assert tensor.shape[0] >= end, f"too many eval_iters"
        sample = tensor[start:end]
        sae_output = sae(sample)
        sae_loss += sae_output['total_loss'].item()
        sae_sparsity_loss += sae_output['sparsity_loss'].item()
        sae_reconstruction_loss += sae_output['reconstruction_loss'].item()
    avg_loss = sae_loss/count
    avg_sparsity_loss = sae_sparsity_loss/count
    avg_reconstruction_loss = sae_reconstruction_loss/count
    return {"reconstruction_loss": avg_reconstruction_loss, "sparsity_loss": avg_sparsity_loss, "total_loss": avg_loss}
    


estimate_sae_loss(100, load_tensor("residuals/residuals_train_1.pt"))

{'reconstruction_loss': 0.16089461743831635,
 'sparsity_loss': 0.32641203701496124,
 'total_loss': 0.4873066544532776}

In [6]:
train_filepaths = []
val_filepaths = []
for file in os.listdir(f'residuals'):
    if file.startswith(f"residuals_train"):
        train_filepaths.append(f"residuals/{file}")
    elif file.startswith(f"residuals_val"):
        val_filepaths.append(f"residuals/{file}")



In [9]:
optimizer = torch.optim.Adam(sae.parameters(), lr=sae_learning_rate)
num_epochs = 1
logging_interval = 50000

for epoch in range(num_epochs):
    for filepath in train_filepaths:
        val_residuals_tensor = load_tensor(random.choice(val_filepaths))
        print(f"val loss on next datafile")
        print(estimate_sae_loss(1000, val_residuals_tensor))
        del val_residuals_tensor
        residuals_tensor = load_tensor(filepath)
        print(f"train loss on next datafile")
        print(estimate_sae_loss(1000, residuals_tensor))
        print(f"training on {filepath}")

        for i in tqdm.tqdm(range(0, residuals_tensor.shape[0]-B, B)):
            start = i
            end = i+B
            assert residuals_tensor.shape[0] >= end, f"too many train samples"
            sample = residuals_tensor[start:end]
            optimizer.zero_grad()
            sae_output = sae(sample)
            sae_reconstruction_loss = sae_output['reconstruction_loss']
            sae_sparsity_loss = sae_output['sparsity_loss']
            total_loss = sae_reconstruction_loss + sae_sparsity_loss
            total_loss.backward()
            optimizer.step()
            if i % logging_interval == 0:
                print(f"reconstruction loss: {sae_reconstruction_loss}, sparsity loss: {sae_sparsity_loss}, total loss: {total_loss}")
            
            


val loss on next datafile
{'reconstruction_loss': 3.5548341926187274e-05, 'sparsity_loss': 0.00010388851584866643, 'total_loss': 0.0001394368577748537}
train loss on next datafile
{'reconstruction_loss': 3.75913274474442e-05, 'sparsity_loss': 9.729796694591642e-05, 'total_loss': 0.00013488929532468318}
training on residuals/residuals_train_5.pt


  0%|          | 51/51187 [00:00<01:40, 508.82it/s]

reconstruction loss: 0.1483488380908966, sparsity loss: 0.3810155689716339, total loss: 0.5293644070625305


  6%|▋         | 3282/51187 [00:06<00:35, 1336.69it/s]

reconstruction loss: 0.14869186282157898, sparsity loss: 0.3963930606842041, total loss: 0.5450849533081055


...

reconstruction loss: 0.16866004467010498, sparsity loss: 0.3251166343688965, total loss: 0.49377667903900146


 98%|█████████▊| 50245/51187 [00:47<00:00, 1373.90it/s]

reconstruction loss: 0.14468492567539215, sparsity loss: 0.287036657333374, total loss: 0.431721568107605


  0%|          | 0/60000 [00:00<?, ?it/s]


NameError: name 'get_batch' is not defined

In [10]:
# Save the model weights
torch.save(sae.state_dict(), 'sae_model_weights.pth')
